# Mini Project 2: IMDB Sentiment Analysis

February 22, 2019

Akshal Aniche, Jacob Sanz-Robinson, Raphael Hotter

COMP 551

## Imports

In [2]:
import numpy as np

## Load the data

In [3]:
import os
pos_train_files = os.listdir('data/train/pos/')
neg_train_files = os.listdir('data/train/neg/')
test_files = os.listdir('data/test/')

pos_words = 'opinion-lexicon-English/positive-words.txt'
neg_words = 'opinion-lexicon-English/negative-words.txt'


# Remove .DS_Store files
while '.DS_Store' in pos_train_files:
  pos_train_files.remove('.DS_Store')
while '.DS_Store' in neg_train_files:
  neg_train_files.remove('.DS_Store')
while '.DS_Store' in test_files:
  test_files.remove('.DS_Store')

test_files.sort(key=lambda x : int(x[:-4]))

In [4]:
# Reads data into 2 arrays
train_text, test_raw = [], []

for file in pos_train_files:
  with open('data/train/pos/{}'.format(file), 'r') as f:
    train_text.append(f.read().lower())

for file in neg_train_files:
  with open('data/train/neg/{}'.format(file), 'r') as f:
    train_text.append(f.read().lower())

for file in test_files:
  with open('data/test/{}'.format(file), 'r') as f:
    test_raw.append(f.read().lower())

In [5]:
# Training set Y vector
pos_goal = np.ones((12500))
neg_goal = np.zeros((12500))
Y_train = np.append(pos_goal, neg_goal, axis = 0)

### Read lexicon of sentiment words

In [6]:
sent_words = []

with open(pos_words, 'r', encoding="latin-1") as f:
    for line in f:
        if (line[0] != ';'):
            sent_words.append(line.strip())

with open(neg_words, 'r', encoding="latin-1") as f:
    for line in f:
        if (line[0] != ';'):
            sent_words.append(line.strip())
            
# print(sent_words)

## Data processer preparation

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import Normalizer

bin_vec = CountVectorizer(binary=True)
tfidf_vec = TfidfVectorizer()
bigram_vec = CountVectorizer(ngram_range=(1,2))
sentc_vec = CountVectorizer(binary=True)
sentt_vec = TfidfVectorizer()

In [36]:
from nltk import word_tokenize, sent_tokenize        
from nltk.stem import WordNetLemmatizer 

#Lemmatizer integrated with the tokenizer 
wnl = WordNetLemmatizer()
class LemmaTokenizer(object):
    def __call__(self, text):
        return [wnl.lemmatize(t) for t in word_tokenize(sent_tokenize(text))]

sentt_vec = TfidfVectorizer(tokenizer=LemmaTokenizer())
tokenizer=LemmaTokenizer()

## Text processing

### Binary representation

In [13]:
# Tokenize and build the vocabulary
bin_vec.fit(train_text)

# Create the feautre matrices 
train_bin = bin_vec.transform(train_text)
test_bin = bin_vec.transform(test_raw)

### Monograms and Bigrams 

In [37]:
# Tokenize and build the vocabulary
bigram_vec.fit(train_text)

# Create the feautre matrices 
train_bigram = bigram_vec.transform(train_text)
test_bigram = bigram_vec.transform(test_raw)

### Tf-Idf 

In [55]:
# Tokenize and build the vocabulary
tfidf_vec.fit(train_text)

# Create the feautre matrices 
train_tfidf = tfidf_vec.fit_transform(train_text)
test_tfidf = tfidf_vec.transform(test_raw)

### Tf-Idf with only sentiment lexicon, Normalized

In [ ]:
# Tokenize and build the vocabulary
sentt_vec.fit(sent_words)

# Create the feautre matrices 
train_s_tfidf = sentt_vec.transform(train_text)
test_s_tfidf = sentt_vec.transform(test_raw)

normalizer_tran = Normalizer().fit(X=train_s_tfidf)
X_train_sn = normalizer_tran.transform(train_s_tfidf)
X_test_sn = normalizer_tran.transform(test_s_tfidf)

KeyboardInterrupt: 

### Normalized Tf-Idf

In [ ]:
from sklearn.preprocessing import Normalizer, PolynomialFeatures
from sklearn.decomposition import TruncatedSVD

def process (X_train, X_test, p, var):
    poly = PolynomialFeatures(p)
    pca = TruncatedSVD(n_components = 10000)
    normalizer_tranformer = Normalizer().fit(X_train)
    X_train = normalizer_tranformer.transform(X_train)
    X_test= normalizer_tranformer.transform(X_test)


    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)
    '''
    poly.fit(X=X_train)
    X_train = poly.transform(X_train)
    X_test = poly.transform(X_test)
    
    pca.fit(X=X_train)
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)
    '''
    return (X_train, X_test)

In [ ]:
(X_train_normalized, X_test_normalized) = process(train_tfidf, test_tfidf, 2, 0.95)
print (X_train_normalized.shape)

In [12]:
from sklearn.preprocessing import Normalizer
normalizer_tranformer = Normalizer().fit(X=train_tfidf)
X_train_normalized = normalizer_tranformer.transform(train_tfidf)
X_test_normalized = normalizer_tranformer.transform(test_tfidf)

## Bernoulli Naive Bayes model from scratch

In [23]:
import math

class NaiveBayesScratch():
    """Bernouli Naive Bayes"""
    def train(self, X_train, Y_train): #bow = bag of words
#         X_train = X_train.toarray()
        Y_train = Y_train.reshape(len(Y_train),1)
        num_pos = np.sum(Y_train)
        num_neg = len(Y_train) - num_pos
        
        self.theta_1_ = num_pos/float(len(Y_train))
        self.theta_j1_ = (X_train*Y_train).sum(axis=0) # sum along training examples
        self.theta_j0_ = (X_train*(1-Y_train)).sum(axis=0)
        
        # Laplace smoothing
        self.theta_j1_ = (self.theta_j1_ + 1)/(float(num_pos) + 2)
        self.theta_j0_ = (self.theta_j0_ + 1)/(float(num_neg) + 2)
        
        # Prepare for predictions
        self.predict_theta_1_ = math.log(self.theta_1_/(1-self.theta_1_))

        self.predict_pos_ = np.log(self.theta_j1_/self.theta_j0_)
        self.predict_pos_ = self.predict_pos_.reshape(len(self.predict_pos_),1)
        
        self.predict_neg_ = np.log((1-self.theta_j1_)/(1-self.theta_j0_))
        self.predict_neg_ = self.predict_neg_.reshape(len(self.predict_neg_),1)
    
    def predict(self, X):
#         X = X.toarray()
        pos = np.dot(X, self.predict_pos_)
        neg = np.dot(1-X, self.predict_neg_)
        delta_predictions = self.predict_theta_1_ + pos + neg
        binary_predictions = delta_predictions > 0
        return binary_predictions

In [15]:
naive_bayes = NaiveBayesScratch()
naive_bayes.train(train_bin, Y_train)

KeyboardInterrupt: 

In [ ]:
mini_X_test = train_bin
mini_Y_test = Y_train
mini_Y_test = mini_Y_test.reshape(mini_Y_test.shape[0],1)

predictions = naive_bayes.predict(mini_X_test)
# print(predictions==mini_Y_test)
score1 = np.sum(predictions==mini_Y_test, axis=0)/float(mini_Y_test.shape[0])
print(score1)

## Cross Validation

### To test standard Naive Bayes:

In [26]:
# Define train(), evaluate() functions
import random
import math

def k_cross_validate(X_train, Y_train, k):
    """
    X_train: n x m array
    Y_train: n x 1 array
    k: number of folds
    """
    indeces = random.sample(range(Y_train.shape[0]), Y_train.shape[0])
    step = int(X_train.shape[0]/k)
    scores = []
    for k_fold in range(k):
        k_validate_indeces = indeces[k_fold*step:(k_fold+1)*step]
        k_train_indeces = [i for i in range(X_train.shape[0]) if i not in k_validate_indeces]
        
        b = NaiveBayesScratch()
        b.train(X_train[k_train_indeces,:], Y_train[k_train_indeces,:])
        predictions = b.predict(X_train[k_validate_indeces,:])
        score = np.sum(predictions==Y_train[k_validate_indeces,:]) / float(Y_train[k_validate_indeces,:].shape[0])
        scores.append(score)
    mean = np.array(scores).mean()
    print("Scores: {}".format(scores))
    print("Scores Mean: {}".format(mean))

    
formatted_Y_train = Y_train.reshape(Y_train.shape[0], 1)
k_cross_validate(train_bin.toarray(), formatted_Y_train, 3)

Scores: [0.858634345373815, 0.842913716548662, 0.8471138845553822]
Scores Mean: 0.8495539821592865


### Cross validation function for sklearn models

In [26]:
from sklearn.model_selection import cross_val_score
# Function to cross validate a scikitlearn model
def crossvalidate(model, X_train, fold) :
    scores = cross_val_score(model, X_train, Y_train, cv=fold)
    mean = scores.mean()
    print("Scores: {}".format(scores))
    print("Scores Mean: {}".format(mean))

## Models

### Naive Bayes (to make sure it worked)

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bayes_clf = BernoulliNB()
crossvalidate(bayes_clf, X_train_normalized, Y_train, 4)

<<<<<<< local


Scores: [0.79552 0.79632 0.79984 0.79904]
Scores Mean: 0.79768


>>>>>>> remote
<<<<<<< local <modified: >


Scores: [0.85552 0.85424 0.832   0.85968]
Scores Mean: 0.85036


>>>>>>> remote <removed>


### Logistic Regression

In [ ]:
from sklearn import linear_model
regr_clf = linear_model.LogisticRegression(solver='lbfgs')
crossvalidate(regr_clf, X_train_normalized, Y_train, 4)

<<<<<<< local


Scores: [0.89008 0.8904  0.87408 0.888  ]
Scores Mean: 0.88564


Scores: [0.86608 0.85936 0.8624  0.86656]
Scores Mean: 0.8635999999999999


>>>>>>> remote


### Decision Trees

In [ ]:
from sklearn import tree
dec_clf = tree.DecisionTreeClassifier()
crossvalidate(dec_clf, X_train_normalized, Y_train, 4)

<<<<<<< local


Scores: [0.70992 0.70832 0.69872 0.69744]
Scores Mean: 0.7035999999999999


KeyboardInterrupt: 

>>>>>>> remote


I wonder if we can play with the parameters more 

### Support Vector Machines

In [ ]:
from sklearn import svm
svm_clf = svm.LinearSVC()
crossvalidate(svm_clf, X_train_normalized, 4)